In [26]:
import pandas as pd

In [106]:
bart_data = pd.read_csv('bartData_7.csv')

In [107]:
bart_data = bart_data[['daypart','Weekpart','Origin','Dest','Count','lat','lon','Name']]

In [108]:
#bart_data.set_index(['Origin', 'Dest','daypart','Weekpart'], drop=True, inplace=True)

bart_data.head(15)

,daypart,Weekpart,Origin,Dest,Count,lat,lon,Name
0,earlyMorning,Weekday,16TH,12TH,2,37.802593,-122.271958,12TH ST/OAKLAND CITY CENTER
1,eveningPeak,Weekday,16TH,12TH,28,37.802593,-122.271958,12TH ST/OAKLAND CITY CENTER
2,midDay,Weekday,16TH,12TH,10,37.802593,-122.271958,12TH ST/OAKLAND CITY CENTER
3,morningPeak,Weekday,16TH,12TH,26,37.802593,-122.271958,12TH ST/OAKLAND CITY CENTER
4,night,Weekday,16TH,12TH,10,37.802593,-122.271958,12TH ST/OAKLAND CITY CENTER
5,earlyMorning,Weekend,16TH,12TH,1,37.802593,-122.271958,12TH ST/OAKLAND CITY CENTER
6,eveningPeak,Weekend,16TH,12TH,13,37.802593,-122.271958,12TH ST/OAKLAND CITY CENTER
7,midDay,Weekend,16TH,12TH,8,37.802593,-122.271958,12TH ST/OAKLAND CITY CENTER
8,morningPeak,Weekend,16TH,12TH,4,37.802593,-122.271958,12TH ST/OAKLAND CITY CENTER
9,night,Weekend,16TH,12TH,9,37.802593,-122.271958,12TH ST/OAKLAND CITY CENTER


In [109]:
bart_dict = {}

In [78]:
#loop for destination data
station = ""
day = ""
days = []
hour = ""
dest = ""

for index,row in bart_data.iterrows():
    if row['Origin'] == station:
        continue
    station = row['Origin']
    latitude = row['lat']
    longitude = row['lon']
    name = row['Name']
    
    bart_dict[station + "HERE"] = {"type": "Feature", "geometry": {"type": "Point","coordinates": [longitude, latitude]}}
    
    try:
        bart_dict[station + "HERE"]["properties"].update({"Code":station, "Name": name, "geometry":{'lon':longitude, 'lat': latitude}, row['Weekpart']: {row['daypart']:{row['Dest']:row['Count']}}})
    except: 
        bart_dict[station + "HERE"]["properties"] = {"Code":station, "Name": name, "geometry":{'lon':longitude, 'lat': latitude}, row['Weekpart']: {row['daypart']:{row['Dest']:row['Count']}}}
    
    station_data = bart_data.loc[bart_data['Origin'] == station]
    
    for index2,row2 in station_data.iterrows():
        if row2['Weekpart'] in days:
            continue
            
        days.append(row2['Weekpart'])
        day = row2['Weekpart']
        try:
            bart_dict[station + "HERE"]["properties"][day].update({row2['daypart']:{row2['Dest']:row2['Count']}})
        except:
            bart_dict[station + "HERE"]["properties"][day] = {row2['daypart']:{row2['Dest']:row2['Count']}}
        
        day_data = station_data.loc[(station_data['Origin'] == station) & (station_data['Weekpart'] == day)]
        
        for index3,row3 in day_data.iterrows():
            if row3['daypart'] == hour:
                continue
            
            hour = row3['daypart']
        
            try:
                bart_dict[station + "HERE"]["properties"][day][hour].update({row3['Dest']:row3['Count']})
            except:
                bart_dict[station + "HERE"]["properties"][day][hour] = {row3['Dest']:row3['Count']}
                
            hour_data = day_data.loc[(day_data['Origin'] == station) & (day_data['Weekpart'] == day) & (day_data['daypart'] == hour)]
            
            for index4, row4 in hour_data.iterrows():
                if row4['Dest'] == dest:
                    continue
                
                dest = row4['Dest']
                
                
                try:
                    bart_dict[station + "HERE"]["properties"][day][hour][dest].update(row4['Count'])
                except:
                    bart_dict[station + "HERE"]["properties"][day][hour][dest] = row4['Count']
                    
    days = []
    hour = ""
    dest = ""
    
                

In [110]:
#loop for origin data
station = ""
stations = []
day = ""
days = []
hour = ""
hours = []
orig = ""
origins = []

for index,row in bart_data.iterrows():
    if row['Dest'] in stations:
        continue
    
    stations.append(row['Dest'])
    station = row['Dest']
    latitude = row['lat']
    longitude = row['lon']
    name = row['Name']
    
    bart_dict[station + "HERE"] = {"type": "Feature", "geometry": {"type": "Point","coordinates": [longitude, latitude]}}
    
    try:
        bart_dict[station + "HERE"]["properties"].update({"Code":station, "Name": name, "geometry":{'lon':longitude, 'lat': latitude}, row['Weekpart']: {row['daypart']:{row['Origin']:row['Count']}}})
    except: 
        bart_dict[station + "HERE"]["properties"] = {"Code":station, "Name": name, "geometry":{'lon':longitude, 'lat': latitude}, row['Weekpart']: {row['daypart']:{row['Origin']:row['Count']}}}
    
    station_data = bart_data.loc[bart_data['Dest'] == station]
    
    for index2,row2 in station_data.iterrows():
        if row2['Weekpart'] in days:
            continue
            
        days.append(row2['Weekpart'])
        day = row2['Weekpart']
        try:
            bart_dict[station + "HERE"]["properties"][day].update({row2['daypart']:{row2['Origin']:row2['Count']}})
        except:
            bart_dict[station + "HERE"]["properties"][day] = {row2['daypart']:{row2['Origin']:row2['Count']}}
        
        day_data = station_data.loc[(station_data['Dest'] == station) & (station_data['Weekpart'] == day)]
        
        for index3,row3 in day_data.iterrows():
            if row3['daypart'] in hours:
                continue
            hours.append(row3['daypart'])
            hour = row3['daypart']
        
            try:
                bart_dict[station + "HERE"]["properties"][day][hour].update({row3['Origin']:row3['Count']})
            except:
                bart_dict[station + "HERE"]["properties"][day][hour] = {row3['Origin']:row3['Count']}
                
            hour_data = day_data.loc[(day_data['Dest'] == station) & (day_data['Weekpart'] == day) & (day_data['daypart'] == hour)]
            
            for index4, row4 in hour_data.iterrows():
                if row4['Origin'] in origins:
                    continue
                
                origins.append(row4['Origin'])
                orig = row4['Origin']
                
                
                try:
                    bart_dict[station + "HERE"]["properties"][day][hour][orig].update(row4['Count'])
                except:
                    bart_dict[station + "HERE"]["properties"][day][hour][orig] = row4['Count']
                
            origins = []
        hours = []         
    days = []
    stations = []
    hour = ""
    orig = ""
    #bart_data = bart_data.loc[bart_data['Origin'] != station]
    
                

In [19]:
import json

In [104]:
bart_dict

{'12THHERE': {'geometry': {'coordinates': [-122.294898766556,
    37.8045800580893],
   'type': 'Point'},
  'properties': {'Code': '12TH',
   'Name': 'WEST OAKLAND',
   'Weekday': {'earlyMorning': {'16TH': 2,
     '19TH': 1,
     '24TH': 2,
     'ASHB': 1,
     'BALB': 2,
     'BAYF': 8,
     'CAST': 5,
     'CIVC': 6,
     'COLM': 9,
     'COLS': 5,
     'CONC': 13,
     'DALY': 5,
     'DBRK': 2,
     'DELN': 34,
     'DUBL': 12,
     'EMBR': 7,
     'FRMT': 11,
     'FTVL': 4,
     'GLEN': 3,
     'HAYW': 8,
     'LAFY': 7,
     'LAKE': 1,
     'MCAR': 3,
     'MLBR': 2,
     'MONT': 6,
     'NBRK': 2,
     'NCON': 7,
     'ORIN': 4,
     'PHIL': 16,
     'PITT': 26,
     'PLZA': 5,
     'POWL': 4,
     'RICH': 5,
     'ROCK': 3,
     'SANL': 11,
     'SBRN': 1,
     'SFIA': 1,
     'SHAY': 7,
     'SSAN': 2,
     'UCTY': 6,
     'WCRK': 10,
     'WDUB': 8,
     'WOAK': 1},
    'eveningPeak': {'16TH': 28,
     '19TH': 3,
     '24TH': 15,
     'ASHB': 7,
     'BALB': 6,
     'BAYF': 

In [111]:
with open('bart_result3.json', 'w') as fp:
    json.dump(bart_dict, fp)